In [1]:
import numpy as np
import geopandas as gpd
from scipy.spatial import Voronoi, voronoi_plot_2d
from shapely.geometry import Polygon, Point
import matplotlib.pyplot as plt
import time


def get_voronoi_polygons(data_df, latitude="latitude", longitude="longitude"):
    """
    Create a list of Voronoi polygons from a list of points
    Args
        data_df: dataframe containing lat/long
        latitude: latitude feature
        longitude: longitude feature
    Returns
        Voronoi polygons graph (points, polygons) from the seed points in data_df
        (a scipy.spatial.Voronoi object)
    """
    
    locations_data = np.array(data_df[[latitude, longitude]].astype(float))
    data_voronoi = [[x[1], x[0]] for x in locations_data]
    voronoi_polygons = Voronoi(data_voronoi)
    print(f"Voronoi polygons: {len(voronoi_polygons.points)}")
    return voronoi_polygons


def plot_voronoi_polygons(voronoi_polygons, title, lat_limits, long_limits):
    """
    Plot Voronoi polygons (visualization tool)
    Args
        voronoi_polygons: Voronoi polygons object (a scipy.spatial.Voronoi object)
        title: graph title
        lat_limits: graph latitude (y) limits
        long_limits: graph longitude (x) limits
    Returns
        None
    """
    # do not show the vertices, only show edges and centers
    fig = voronoi_plot_2d(voronoi_polygons,
                     show_vertices=False)
    plt.xlim(long_limits)
    plt.ylim(lat_limits)    
    plt.title(title)
    plt.show()

def extract_voronoi_polygon_list(voronoi_polygons):
    """
    Extract Voronoi polygons list from a Voronoi polygons object
    
    Args
        Voronoi polygons object
    Returns
        list of Voronoi polygons regions
    """
    voronoi_poly_list = []
    for region in voronoi_polygons.regions:
        if -1 in region:
            continue
        else:
            pass
        if len(region) != 0:
            voronoi_poly_region = Polygon(voronoi_polygons.vertices[region])
            voronoi_poly_list.append(voronoi_poly_region)
        else:
            continue
    return voronoi_poly_list

def clip_polygons(poly_list_origin, poly_clipping):
    """
    Clip a list of polygons using an external polygon
    Args:
        poly_list_origin: list of polygons to clip
        poly_clipping: polygon used to clip the original list
    
    Returns:
        The original list of polygons, with the polygons cliped using the clipping polygon
    """
    
    #convert the initial polygons list to a geodataframe
    polygons_gdf = gpd.GeoDataFrame(poly_list_origin, columns = ['geometry'], crs=poly_clipping.crs)
    start_time = time.time()
    polygons_clipped = gpd.clip(polygons_gdf, poly_clipping)
    end_time = time.time()
    print(f"Total time: {round(end_time - start_time, 4)} sec.")
    return polygons_clipped


def within_polygon(data_original_df, polygon, latitude="latitude", longitude="longitude"):
    """
    Args
        data_original_df: dataframe with latitude / longitude
        polygon: polygon (Polygon object)
        latitude: feature name for latitude n data_original_df
        longitude: feature name for longitude in data_original_df
    Returns
        coordinates of points inside polygon
        coordinates of points outside polygon
        polygon transformed into a geopandas dataframe
    """
    data_df = data_original_df.copy()
    data_df["in_poly"] = data_df.apply(lambda x: Point(x[longitude], x[latitude]).within(polygon), axis=1)
    data_in_df = data_df[[longitude, latitude]].loc[data_df["in_poly"]==True]
    data_out_df = data_df[[longitude, latitude]].loc[data_df["in_poly"]==False]
    data_in_df.columns = ["long", "lat"]
    data_out_df.columns = ["long", "lat"]
    sel_polygon_gdf = gpd.GeoDataFrame([polygon], columns = ['geometry'])
    return data_in_df, data_out_df, sel_polygon_gdf

def get_polygons_area(data_gdf):
    """
    Add a column with polygons area to a GeoDataFrame
    A Cylindrical equal area projection is used to calculate 
    polygons area
    
    Args
        data_gdf: a GeoDataFrame
    Returns
        the original data_gdf with an `area` column added
    """
    # copy the data, to not affect initial data projection
    data_cp = data_gdf.copy()
    # transform, in the copied data, the projection in Cylindrical equal-area,
    # which preserves the areas 
    data_cp = data_cp.to_crs({'proj':'cea'})
    data_cp["area"] = data_cp['geometry'].area / 10**6 # km^2
    data_gdf["area"] = data_cp["area"]
    # returns the initial data, with added area columns
    return data_gdf
    

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string
